## Set Up

Choose the MTG set and draft format (Premier or Traditional) to analyze

In [ ]:
# Choose the MTG set. This is the 3 letter code for a set
EXPANSION = "LCI"

FORMAT_PREMIER_DRAFT = "PremierDraft"
FORMAT_TRADITIONAL_DRAFT = "TradDraft"

# Choose the format to analyze
FORMAT = FORMAT_PREMIER_DRAFT

### Download the draft dataset

Download the datasets. The datasets are from the [17 Lands Public Data Sets](https://www.17lands.com/public_datasets).

In [ ]:
GAME_DATA_FILE = f"game_data_public.{EXPANSION}.{FORMAT}.csv.gz"
GAME_DATA_REMOTE_URL = f"https://17lands-public.s3.amazonaws.com/analysis_data/game_data/{GAME_DATA_FILE}"
DRAFT_DATA_FILE = f"draft_data_public.{EXPANSION}.{FORMAT}.csv.gz"
DRAFT_DATA_REMOTE_URL = f"https://17lands-public.s3.amazonaws.com/analysis_data/draft_data/{DRAFT_DATA_FILE}"

!wget {GAME_DATA_REMOTE_URL}
!wget {DRAFT_DATA_REMOTE_URL}
!wget https://17lands-public.s3.amazonaws.com/analysis_data/cards/cards.csv

### Import Pandas and Set Useful Options

Pandas is what we'll use to analyze the data. For more info on Pandas, see the [docs](https://pandas.pydata.org/docs/user_guide/10min.html).

In [ ]:
import gzip
import pandas as pd

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 1600)

### Explore The Data Set

There are 3 key datasets to explore. The main datasets are the draft and game data sets for an MTG set. Another dataset that can often be useful is the list of all cards on arena.

In [ ]:
# Game Data
df = next(pd.read_csv(GAME_DATA_FILE, chunksize=100))
df.head(25)

In [ ]:
# Draft Data
df = next(pd.read_csv(DRAFT_DATA_FILE, chunksize=100))
df.head(55)

In [ ]:
cards_table = pd.read_csv("cards.csv")
set_cards = cards_table[cards_table["expansion"] == EXPANSION]
set_cards.head(25)

## Analyze Data

### Basic Helper Functions

Some useful functions for common ways to use the data sets

In [ ]:
def get_all_drafts():
  """
  Returns a table with one row per draft
  """
  cols = ["expansion", "event_type", "draft_id", "draft_time", "rank", "event_match_wins", "event_match_losses"]
  chunks = list()
  for draft_data in pd.read_csv(
      DRAFT_DATA_FILE,
      chunksize=100000,
      usecols=cols
      ):
    draft_data_no_dups = draft_data.drop_duplicates(subset=["draft_id"])
    chunks.append(draft_data_no_dups)

  all_drafts = pd.concat(chunks)

  # Remove duplicates in case of drafts that show up in multiple chunks
  all_drafts = all_drafts.drop_duplicates(subset=["draft_id"], keep="last")
  return all_drafts

In [ ]:
def get_game_data_cols():
  """
  Returns the columns in the game data file that includes metadata and which cards were in the deck. Filters out other columns to reduce size of the dataset
  """
  df = next(pd.read_csv(GAME_DATA_FILE, chunksize=100))
  col_names = list(df)
  gd_card_cols = [x for x in col_names if x.startswith("deck_")]

  gd_base_cols = ['draft_id', 'main_colors', 'splash_colors', 'user_n_games_bucket', 'user_game_win_rate_bucket', 'won']
  gd_all_cols = gd_base_cols + gd_card_cols
  return gd_all_cols

def get_all_decks(summerize):
  """
  summerize: if True, only return the last deck for each draft
  Returns all decks available in the format.
  """
  gd_all_cols = get_game_data_cols()
  chunks = list()
  for game_data in pd.read_csv(
      GAME_DATA_FILE,
      chunksize=100000,
      usecols=gd_all_cols
      ):
    if summerize:
      # Drop duplciates on draft id, keep the last
      game_data_no_dups = game_data.drop_duplicates(subset=["draft_id"], keep="last")
      chunks.append(game_data_no_dups)
    else:
      chunks.append(game_data)

  all_games = pd.concat(chunks)
  if summerize:
    all_games = all_games.drop_duplicates(subset=["draft_id"], keep="last")
  all_games_names_fixed = all_games.rename(columns=lambda x: x[5:] if x.startswith("deck_") else x)
  return all_games_names_fixed

### Caves Decks

Let's enrich the dataset with counts of relevant cave & cave payoff cards

In [ ]:
hidden_caves = ["Hidden Cataract", "Hidden Courtyard", "Hidden Necropolis", "Hidden Nursery", "Hidden Volcano"]
flip_caves = ["Brass's Tunnel-Grinder", "Dowsing Device", "Grasping Shadows", "Tarrian's Journal", "Twists and Turns"]
fixing_caves = ["Captivating Cave", "Forgotten Monument", "Promising Vein", "Sunken Citadel"]
non_flip_cave_cards = ["Captivating Cave", "Cavernous Maw", "Echoing Deeps", "Forgotten Monument", "Hidden Cataract", "Hidden Courtyard", "Hidden Necropolis", "Hidden Nursery", "Hidden Volcano", "Pit of Offerings", "Promising Vein", "Sunken Citadel", "Volatile Fault"]

cave_payoffs = ["Bat Colony", "Calamitous Cave-In", "Gargantuan Leech", "Sinuous Benthisaur"]
cave_related = ["Compass Gnome", "Cosmium Confluence", "Glimpse the Core", "Kaslem's Stonetree", "Scampering Surveyor", "Spelunking"]

def add_cave_data(decks):
  caves_decks = decks.copy()

  caves_decks["hidden_caves_count"] = caves_decks[hidden_caves].sum(axis=1)
  caves_decks["flip_caves"] = caves_decks[flip_caves].sum(axis=1)
  caves_decks["fixing_caves"] = caves_decks[fixing_caves].sum(axis=1)
  caves_decks["non_flip_cave_cards"] = caves_decks[non_flip_cave_cards].sum(axis=1)
  caves_decks["non_flip_cave_cards"] = caves_decks.apply(lambda x: min(x["non_flip_cave_cards"], 11), axis=1)

  caves_decks["cave_payoffs"] = caves_decks[cave_payoffs].sum(axis=1)
  caves_decks["cave_payoffs"] = caves_decks.apply(lambda x: min(x["cave_payoffs"], 7), axis=1)
  caves_decks["cave_related"] = caves_decks[cave_related].sum(axis=1)

  return caves_decks

In [ ]:
# This step may take a few minutes
USE_WIN_PERCENT = True

if USE_WIN_PERCENT:
  all_decks = get_all_decks(summerize = False)
else:
  all_decks = get_all_decks(summerize = True)

draft_metadata = get_all_drafts()
all_decks.head(25)

In [ ]:
def get_cave_aggregations(use_win_percentage, decks, draft_metadata):
  win_col_name = "won_int" if use_win_percentage else "event_match_wins"
  df = decks.copy()

  if use_win_percentage:
    df["won_int"] = df["won"].astype(int)
  else:
    df = pd.merge(decks, draft_metadata, on=["draft_id"])

  df = df.filter(items=["hidden_caves_count", "flip_caves", "fixing_caves", "non_flip_cave_cards", "cave_payoffs", "cave_related", win_col_name])
  df = df.groupby(["non_flip_cave_cards", "cave_payoffs"]).agg(["mean", "size"])

  df = df[win_col_name].copy()
  df = df.reset_index()
  return df

In [ ]:
all_decks_with_cave_data = add_cave_data(all_decks)
cave_stats = get_cave_aggregations(USE_WIN_PERCENT, all_decks_with_cave_data, draft_metadata)
cave_stats.head(25)

### Graphs

Now that the data has been enriched, we can graph the data with a heatmap. Note, we only show boxes that have enough data points to be relevant

In [ ]:
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def graph_cave_data(cave_decks, max_cave_payoffs, max_caves, use_win_percent):
  hm = np.zeros((max_cave_payoffs, max_caves))
  hm.fill(None)
  size_threshold = 50 if use_win_percent else 10

  for index, row in cave_decks.iterrows():
    non_flip_cave_cards = int(row["non_flip_cave_cards"])
    cave_payoffs = int(row["cave_payoffs"])
    mean = row["mean"]
    if row["size"] >= size_threshold:
      hm[cave_payoffs, non_flip_cave_cards] = mean

  hm_df = DataFrame(hm, index=range(max_cave_payoffs), columns=range(max_caves))
  hm_df = hm_df[::-1]

  plt.figure(figsize=(16,9))
  sns.set(style="ticks")
  plt.style.use("dark_background")

  vmin = .45 if use_win_percent else 2
  vmax = .59 if use_win_percent else 3
  center = .52 if use_win_percent else 2.5

  ax = sns.heatmap(
      hm_df,
      annot=True,
      cmap="RdYlGn",
      robust=True,
      fmt=".2f",
      linewidth=.25,
      linecolor="black",
      center=center,
      vmin=vmin,
      vmax=vmax,
      cbar=False
      )
  ax.set(xlabel="Caves in Deck", ylabel="Cave Payoffs in Deck")
  ax.xaxis.labelpad = 20
  ax.yaxis.labelpad = 20
  return ax

In [ ]:
ax = graph_cave_data(cave_stats, 8, 12, USE_WIN_PERCENT)

### Other Stats

Other stats I was looking at during development. These may need tweaking, as they were originally built without using win percentage.

In [ ]:
optimal_cave_decks = all_decks_with_cave_data[(all_decks_with_cave_data["non_flip_cave_cards"] >= 8) & (all_decks_with_cave_data["cave_payoffs"] >= 4)]
print(len(optimal_cave_decks))
# optimal_cave_decks.head(25)


color_group_breakdown = optimal_cave_decks.copy()
color_group_breakdown = color_group_breakdown.filter(items=["main_colors"])
cgb_group = color_group_breakdown.value_counts()
print(cgb_group)

cgb = optimal_cave_decks.copy()
cgb = cgb.filter(items=["main_colors", "splash_colors"])
cgb["all_colors"] = cgb['main_colors'].str.cat(cgb[['splash_colors']], na_rep='')
for c in "WUBRG":
  print(f"Checking color: {c}")
  cgb[c] = cgb["all_colors"].str.contains(c)
  print(cgb[c].value_counts(normalize=True).mul(100).astype(str)+'%')

cgb["num_colors"] = cgb["all_colors"].str.len()
print(cgb["num_colors"].value_counts(normalize=True))
print(cgb)


cave_cats = [hidden_caves, flip_caves, fixing_caves, non_flip_cave_cards, cave_payoffs, cave_related]
for cc in cave_cats:
  cave_copy = optimal_cave_decks.copy()
  cc_stats = cave_copy.filter(items=cc).mean().sort_values(ascending=False)
  print("\n\n")
  print(cc_stats)